<a href="https://colab.research.google.com/github/ThousandAI/Application-of-AI/blob/main/class09/transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[Transformers Hugging Face](https://huggingface.co/docs/transformers/index)**

In [1]:
!pip install transformers
from transformers import pipeline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.9 MB/s 
     |████████████████████████████████| 101 kB 14.0 MB/s 
     |████████████████████████████████| 6.6 MB 57.2 MB/s 
     |████████████████████████████████| 596 kB 71.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## **Pipeline**

In [2]:
pipe = pipeline("text-classification")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [3]:
pipe("Even if the AI ​​course is difficult, we still enjoy the course content brought to us by the teacher.")

[{'label': 'POSITIVE', 'score': 0.9997095465660095}]

In [4]:
pipe("Code is more difficult to understand than other courses.")

[{'label': 'NEGATIVE', 'score': 0.9992061257362366}]

In [24]:
pipe(["Even if the AI ​​course is difficult, we still enjoy the course content brought to us by the teacher",
      "Code is more difficult to understand than other courses."])

[{'label': 'POSITIVE', 'score': 0.9996920824050903},
 {'label': 'NEGATIVE', 'score': 0.9992061257362366}]

In [6]:
generator = pipeline(task="text-generation")

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [9]:
print(generator("It's annoying to go out in the rain"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 50 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


[{'generated_text': 'It\'s annoying to go out in the rain because you lose ground and then fall off a cliff," he said. "So we made sure that we kept ourselves on top like we did once in the winter when you go in the rain. Our team'}]


In [12]:
vision_classifier = pipeline(task="image-classification")

No model was supplied, defaulted to google/vit-base-patch16-224 and revision 5dca96d (https://huggingface.co/google/vit-base-patch16-224).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [19]:
preds = vision_classifier(
    images="https://images.unsplash.com/photo-1561731216-c3a4d99437d5?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=928&q=80"
)

In [20]:
preds

[{'label': 'tiger, Panthera tigris', 'score': 0.8916527032852173},
 {'label': 'tiger cat', 'score': 0.10138965398073196},
 {'label': 'jaguar, panther, Panthera onca, Felis onca',
  'score': 0.001509590190835297},
 {'label': 'lion, king of beasts, Panthera leo',
  'score': 0.001039321068674326},
 {'label': 'leopard, Panthera pardus', 'score': 0.0007997617358341813}]

In [23]:
from transformers import BertTokenizer, GPT2LMHeadModel, TextGenerationPipeline
tokenizer = BertTokenizer.from_pretrained("uer/gpt2-chinese-cluecorpussmall")
model = GPT2LMHeadModel.from_pretrained("uer/gpt2-chinese-cluecorpussmall")
text_generator = TextGenerationPipeline(model, tokenizer) 
text_generator("今天想去吃拉麵", max_length=50, do_sample=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': '今天想去吃拉麵 就 点 了 一 份 套 餐 ， 好 好 吃 ， 湯 也 很 棒 ， 不 過 一 份 太 少 了 ， 不 夠 吃 很 多 的. 另 外 叫 了 一 份 海 鮮 飯 ， 里'}]

## **English Model**

In [25]:
english_model_name = "distilbert-base-uncased-finetuned-sst-2-english"

In [26]:
from transformers import AutoTokenizer

In [27]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=english_model_name)

In [28]:
tokenizer.tokenize("Even if the AI ​​course is difficult, we still enjoy the course content brought to us by the teacher.")

['even',
 'if',
 'the',
 'ai',
 'course',
 'is',
 'difficult',
 ',',
 'we',
 'still',
 'enjoy',
 'the',
 'course',
 'content',
 'brought',
 'to',
 'us',
 'by',
 'the',
 'teacher',
 '.']

In [29]:
tokenizer("Even if the AI ​​course is difficult, we still enjoy the course content brought to us by the teacher.")

{'input_ids': [101, 2130, 2065, 1996, 9932, 2607, 2003, 3697, 1010, 2057, 2145, 5959, 1996, 2607, 4180, 2716, 2000, 2149, 2011, 1996, 3836, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [31]:
X_train = ["Even if the AI ​​course is difficult, we still enjoy the course content brought to us by the teacher",
           "Code is more difficult to understand than other courses."]

batch = tokenizer(X_train, padding=True, truncation=True, max_length=512, return_tensors="pt")
print(batch)

{'input_ids': tensor([[ 101, 2130, 2065, 1996, 9932, 2607, 2003, 3697, 1010, 2057, 2145, 5959,
         1996, 2607, 4180, 2716, 2000, 2149, 2011, 1996, 3836,  102],
        [ 101, 3642, 2003, 2062, 3697, 2000, 3305, 2084, 2060, 5352, 1012,  102,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [32]:
from transformers import AutoModelForSequenceClassification
import torch
import torch.nn as nn
import torch.nn.functional as F

In [33]:
english_model = AutoModelForSequenceClassification.from_pretrained(english_model_name)

In [34]:
with torch.no_grad():
  outputs= english_model(**batch)
  #outputs= model(**batch, labels=torch.tensor([1, 0]))
  print(outputs)
  predictions = F.softmax(outputs.logits, dim=1)
  print(predictions)
  labels = torch.argmax(predictions, dim=1)
  print(labels)
  labels_mapping = [english_model.config.id2label[label_id] for label_id in labels.tolist()]
  print(labels_mapping)

SequenceClassifierOutput(loss=None, logits=tensor([[-3.9319,  4.1532],
        [ 3.9425, -3.1952]]), hidden_states=None, attentions=None)
tensor([[3.0798e-04, 9.9969e-01],
        [9.9921e-01, 7.9389e-04]])
tensor([1, 0])
['POSITIVE', 'NEGATIVE']


## **Chinese Model**

In [35]:
chinese_pipe = pipeline('fill-mask', model="bert-base-chinese", tokenizer="bert-base-chinese")

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [36]:
chinese_pipe("這門課雖然很"+chinese_pipe.tokenizer.mask_token+"但我還是努力將它完成")

[{'score': 0.48670488595962524,
  'sequence': '這 門 課 雖 然 很 難 但 我 還 是 努 力 將 它 完 成',
  'token': 7432,
  'token_str': '難'},
 {'score': 0.10295306146144867,
  'sequence': '這 門 課 雖 然 很 累 但 我 還 是 努 力 將 它 完 成',
  'token': 5168,
  'token_str': '累'},
 {'score': 0.06878484040498734,
  'sequence': '這 門 課 雖 然 很 短 但 我 還 是 努 力 將 它 完 成',
  'token': 4764,
  'token_str': '短'},
 {'score': 0.05155231058597565,
  'sequence': '這 門 課 雖 然 很 苦 但 我 還 是 努 力 將 它 完 成',
  'token': 5736,
  'token_str': '苦'},
 {'score': 0.023980475962162018,
  'sequence': '這 門 課 雖 然 很 忙 但 我 還 是 努 力 將 它 完 成',
  'token': 2564,
  'token_str': '忙'}]

In [38]:
chinese_pipe("今晚訂"+chinese_pipe.tokenizer.mask_token+"已經客滿，明天下午還有空位")

[{'score': 0.49522095918655396,
  'sequence': '今 晚 訂 位 已 經 客 滿 ， 明 天 下 午 還 有 空 位',
  'token': 855,
  'token_str': '位'},
 {'score': 0.37189793586730957,
  'sequence': '今 晚 訂 房 已 經 客 滿 ， 明 天 下 午 還 有 空 位',
  'token': 2791,
  'token_str': '房'},
 {'score': 0.07200019806623459,
  'sequence': '今 晚 訂 單 已 經 客 滿 ， 明 天 下 午 還 有 空 位',
  'token': 1606,
  'token_str': '單'},
 {'score': 0.01261210162192583,
  'sequence': '今 晚 訂 票 已 經 客 滿 ， 明 天 下 午 還 有 空 位',
  'token': 4873,
  'token_str': '票'},
 {'score': 0.011162370443344116,
  'sequence': '今 晚 訂 座 已 經 客 滿 ， 明 天 下 午 還 有 空 位',
  'token': 2429,
  'token_str': '座'}]